In [1]:
import random
import json
import numpy as np
import torch
import math
from nltk.tokenize import word_tokenize, sent_tokenize


In [4]:
word_tokenize("Then and again  , my friend Jeb recently , himself, would have been too moderate , too reasonable for GOP Bush . '' I did meant fault . n't leave the Republican Party ; it was misinterpreted left me  many news organizations did yesterday . Both sides are at Crist 's final farewell speech -- as --  ,  Jaryn Emhof , a spokeswoman. . For sake  , they have n't left . BuzzFeed : Jeb : today 's , Reagan In   : Jeb : Ronald Reagan :  .")

['Then',
 'and',
 'again',
 ',',
 'my',
 'friend',
 'Jeb',
 'recently',
 ',',
 'himself',
 ',',
 'would',
 'have',
 'been',
 'too',
 'moderate',
 ',',
 'too',
 'reasonable',
 'for',
 'GOP',
 'Bush',
 '.',
 '``',
 'I',
 'did',
 'meant',
 'fault',
 '.',
 "n't",
 'leave',
 'the',
 'Republican',
 'Party',
 ';',
 'it',
 'was',
 'misinterpreted',
 'left',
 'me',
 'many',
 'news',
 'organizations',
 'did',
 'yesterday',
 '.',
 'Both',
 'sides',
 'are',
 'at',
 'Crist',
 "'s",
 'final',
 'farewell',
 'speech',
 '--',
 'as',
 '--',
 ',',
 'Jaryn',
 'Emhof',
 ',',
 'a',
 'spokeswoman',
 '.',
 '.',
 'For',
 'sake',
 ',',
 'they',
 'have',
 "n't",
 'left',
 '.',
 'BuzzFeed',
 ':',
 'Jeb',
 ':',
 'today',
 "'s",
 ',',
 'Reagan',
 'In',
 ':',
 'Jeb',
 ':',
 'Ronald',
 'Reagan',
 ':',
 '.']

In [2]:
file_path = "../../results_serialized_val_filtered.jsonl" #Each line is a json

ids_sentscores = []
with open(file_path) as f:
    for test_sample in f:
        ids_sentscores.append(json.loads(test_sample))
print(ids_sentscores[1])
# original_justifications = []
# for samp in ids_sentscores:
#     temp_ls = []
#     temp_dict = {}
#     top_justifications = sorted(samp['sentence_scores'], reverse=True, key = lambda x: x[1])[:6]
    
#     for sent in top_justifications:
#         temp_ls.append(sent[0].strip() + "<>")
    
#     temp_dict["id"] = samp["id"]
#     temp_dict["justifications"] = " ".join(temp_ls)
#     original_justifications.append(temp_dict)
#     print(temp_dict)
    
# json.dump(original_justifications, open('../../top6_val_justifications_sep.json','w+'))

{'id': '238.json', 'sentence_scores': [['An anonymous e-mail says Barack Obama took the oath of office for the U.S. Senate on a Koran, the holy book of Islam.', 0.6007211804389954], ['We thought it would be odd if that were true, since Obama is a Christian.', 0.5864729285240173], ['In fact, it is wrong.', 0.6335600018501282], ['The e-mail also spells the book\'s name "Kuran," though usually it is spelled Koran or Quran.', 0.7251154780387878], ["Two press reports from Obama's swearing-in ceremony in January 2005 mention specifically that Obama took the oath of office by placing his hand on his own copy of the Bible.", 0.7941301465034485], ['The Barack Obama campaign also confirmed that it was a Bible and that the book belonged to Obama.', 0.781171441078186], ['Vice President Dick Cheney, in his role as president of the Senate, administered the oath.', 0.6896082758903503], ['After being raised outside of any particular faith tradition, Obama became a Christian in his mid 20s and is a mem

In [ ]:
file_path = "../../results_serialized_val_filtered.jsonl" #Each line is a json

ids_sentscores = []
with open(file_path) as f:
    for test_sample in f:
        ids_sentscores.append(json.loads(test_sample))
        
top_justifications = sorted(ids_sentscores[0]['sentence_scores'], reverse=True, key = lambda x: x[1])[:6]
print(top_justifications)

In [ ]:
a = "Building a wall on the border? will take literally years. Rick Perry, agog. Perry’s assertion got us thinking"
new_a1 = a.split(".")
new_a2 = str(a.split(".")).split("?")


print(new_a1)
for i in new_a1:
    print(i)
    print("--------")

print(new_a2)
for i in new_a2:
    print(i)
    print("--------")



In [ ]:
# For Debugging of GPT2

# print([tokenizer.decode(idx) for idx in tensor_input.cpu()[0].tolist()])
# print(tensor_input.shape)
# print(logits.shape) #(1, 11, 50257)
# preds = -torch.gather(input=F.softmax(logits[:, :-1,:], dim=2), index=tensor_input[:,1:].unsqueeze(2), dim=2).squeeze().log()
# print(preds)
# print(preds.mean().exp())
# print("ppl", math.exp(loss.item()))
# input()
